# Customer Behaviour Insight Engine - Notebook 6b
## CBS Digital Adoption - Cleaned & Translated (Analysis-Ready)

**Goal of 6b**

Notebook 6a explored the raw CBS dataset about ICT-gebruik van personen (internet & device usage) and kept all original Dutch column names. 

This notebook will: 

- Reload the same CBS CSV from `data/external`
- Select only the columns needed for device / mobile usage
- Translate Dutch column names into clear English ones
- Reshape into a tidy format (`year`, `device-company`, `shape_pct`)
- Save the cleaned, analysis-ready table to: 

`../data/processed/cbs_digital_clean.csv`

In [17]:
import pandas as pd
import numpy as np 

pd.set_option("display.max_columns", 100)
pd.set_option("display.width", 120)
pd.set_option("display.float_format", "{:.2f}".format)

## 1. Load the raw CBS dataet (same source as 6a)

The data comes from CBS Open Data (ICT-gebruik van personen) and was downloaded as a semicolon-separated CSV with coma decimals. 

Here I: 

- Laod the file from `../data/external/cbs_digital_adoption.csv`
- Tell pandas explicitly: 
    - `sep=";"` -> EU-style delimiter
    - `decimal=","` → convert `88,3` to `88.3`
- Keep all Dutch column names as-is for now.



In [18]:
cbs_path = "../data/external/cbs_digital_adoption.csv"

cbs_raw = pd.read_csv(
    cbs_path,
    sep=";",          # semicolon-separated
    decimal=",",      # comma decimals
    encoding="utf-8", # UTF-8 is fine here
)

print("Raw CBS shape:", cbs_raw.shape)
cbs_raw.head()

Raw CBS shape: (8, 38)


,Kenmerken personen,Marges,Perioden,Internet gebruik/wanneer internet het laatst gebruikt/Minder dan 3 maanden geleden (% van personen 12 jaar of ouder),Internet gebruik/wanneer internet het laatst gebruikt/3 tot 12 maanden geleden (% van personen 12 jaar of ouder),Internet gebruik/wanneer internet het laatst gebruikt/Meer dan 12 maanden geleden (% van personen 12 jaar of ouder),Internet gebruik/wanneer internet het laatst gebruikt/Nooit internet gebruikt (% van personen 12 jaar of ouder),Internet gebruik/Frequentie gebruik afgelopen 3 maanden/(Bijna) elke dag (% van personen 12 jaar of ouder),Internet gebruik/Frequentie gebruik afgelopen 3 maanden/Minstens een keer per week (% van personen 12 jaar of ouder),Internet gebruik/Frequentie gebruik afgelopen 3 maanden/Minder dan een keer per week (% van personen 12 jaar of ouder),Internet gebruik/Plaats gebruik afgelopen 3 maanden/Thuis (% van personen 12 jaar of ouder),Internet gebruik/Plaats gebruik afgelopen 3 maanden/Bij iemand anders thuis (% van personen 12 jaar of ouder),Internet gebruik/Plaats gebruik afgelopen 3 maanden/Op het werk (% van personen 12 jaar of ouder),Internet gebruik/Plaats gebruik afgelopen 3 maanden/Bij een onderwijsinstelling (% van personen 12 jaar of ouder),Internet gebruik/Plaats gebruik afgelopen 3 maanden/Ergens anders (% van personen 12 jaar of ouder),Internet gebruik/Mobiel gebruik afgelopen 3 maanden/Totaal (% van personen 12 jaar of ouder),Internet gebruik/Mobiel gebruik afgelopen 3 maanden/Mobiele telefoon of smart phone (% van personen 12 jaar of ouder),Internet gebruik/Mobiel gebruik afgelopen 3 maanden/Laptop (% van personen 12 jaar of ouder),Internet gebruik/Mobiel gebruik afgelopen 3 maanden/Tablet (% van personen 12 jaar of ouder),Internet gebruik/Mobiel gebruik afgelopen 3 maanden/Andere mobiele apparatuur (% van personen 12 jaar of ouder),Internet activiteiten/Communicatie/Versturen/ontvangen e-mails (% van personen 12 jaar of ouder),Internet activiteiten/Informatie en vermaak/Gebruik van diensten in de reisbranche (% van personen 12 jaar of ouder),Internet activiteiten/Informatie en vermaak/Zoeken van baan of solliciteren (% van personen 12 jaar of ouder),Internet activiteiten/Dienstverlening/Onderwijs/Volgen van online cursus (% van personen 12 jaar of ouder),Internet activiteiten/Dienstverlening/Overheids- en publieke instanties/Gebruik websites overheid/Zoeken op websites (% van personen 12 jaar of ouder),Internet activiteiten/Dienstverlening/Overheids- en publieke instanties/Gebruik websites overheid/Officiële documenten downloaden (% van personen 12 jaar of ouder),Internet activiteiten/Dienstverlening/Overheids- en publieke instanties/Gebruik websites overheid/Ingevulde documenten versturen (% van personen 12 jaar of ouder),Internet activiteiten/Dienstverlening/Overheids- en publieke instanties/Gebruik websites publieke instanties/Zoeken op websites (% van personen 12 jaar of ouder),Internet activiteiten/Dienstverlening/Overheids- en publieke instanties/Gebruik websites publieke instanties/Officiële documenten downloaden (% van personen 12 jaar of ouder),Internet activiteiten/Dienstverlening/Overheids- en publieke instanties/Gebruik websites publieke instanties/Ingevulde documenten versturen (% van personen 12 jaar of ouder),Internet activiteiten/Dienstverlening/Gebruik internet als opslagmedium/Cloud computing (% van personen 12 jaar of ouder),"Internet activiteiten/Dienstverlening/Gebruik internet als opslagmedium/Soort van opgeslagen bestanden/Tekst, spreadsheets, presentaties (% van personen 12 jaar of ouder)",Internet activiteiten/Dienstverlening/Gebruik internet als opslagmedium/Soort van opgeslagen bestanden/Foto's (% van personen 12 jaar of ouder),Internet activiteiten/Dienstverlening/Gebruik internet als opslagmedium/Soort van opgeslagen bestanden/E-Books of e-magazins (% van personen 12 jaar of ouder),Internet activiteiten/Dienstverlening/Gebruik internet als opslagmedium/Soort van opgeslagen bestanden/Muziek (% van perso

## 2. Select only the device / mobile usage columns

From Notebook 6a, the relevant columns for device usage are: 

- `Perioden` → year  
- `Internet gebruik/Mobiel gebruik afgelopen 3 maanden/Totaal (...)`
- `Internet gebruik/Mobiel gebruik afgelopen 3 maanden/Mobiele telefoon of smart phone (...)`
- `Internet gebruik/Mobiel gebruik afgelopen 3 maanden/Laptop (...)`
- `Internet gebruik/Mobiel gebruik afgelopen 3 maanden/Tablet (...)`
- `Internet gebruik/Mobiel gebruik afgelopen 3 maanden/Andere mobiele apparatuur (...)`

These represent:

- Overall mobile internet usage
- Smartphone usage
- Laptop usage (as a mobile device)
- Tablet usage
- Other mobile equipment



In [19]:
# Exact device-realted coumn names (from 6a)

device_cols_raw = [
    "Internet gebruik/Mobiel gebruik afgelopen 3 maanden/Totaal (% van personen 12 jaar of ouder)",
    "Internet gebruik/Mobiel gebruik afgelopen 3 maanden/Mobiele telefoon of smart phone (% van personen 12 jaar of ouder)",
    "Internet gebruik/Mobiel gebruik afgelopen 3 maanden/Laptop (% van personen 12 jaar of ouder)",
    "Internet gebruik/Mobiel gebruik afgelopen 3 maanden/Tablet (% van personen 12 jaar of ouder)",
    "Internet gebruik/Mobiel gebruik afgelopen 3 maanden/Andere mobiele apparatuur (% van personen 12 jaar of ouder)",
]

cols_to_keep = ["Perioden"] + device_cols_raw

cbs_devices_wide = cbs_raw[cols_to_keep].copy()
cbs_devices_wide.head()

,Perioden,Internet gebruik/Mobiel gebruik afgelopen 3 maanden/Totaal (% van personen 12 jaar of ouder),Internet gebruik/Mobiel gebruik afgelopen 3 maanden/Mobiele telefoon of smart phone (% van personen 12 jaar of ouder),Internet gebruik/Mobiel gebruik afgelopen 3 maanden/Laptop (% van personen 12 jaar of ouder),Internet gebruik/Mobiel gebruik afgelopen 3 maanden/Tablet (% van personen 12 jaar of ouder),Internet gebruik/Mobiel gebruik afgelopen 3 maanden/Andere mobiele apparatuur (% van personen 12 jaar of ouder)
0,2012,51.40,40.30,29.20,16.10,NaN
1,2013,60.70,53.60,24.30,19.90,1.60
2,2014,65.90,60.40,23.70,23.30,4.90
3,2015,71.10,66.60,24.80,25.20,5.10
4,2016,73.20,70.50,22.00,21.90,4.50


## 3. Translate Dutch column names -> English analysis names 

for analysis and integration with other datasets, I want short, clear English names. 
**Original → New:**

- `Perioden` → `year`
- `Internet gebruik/Mobiel gebruik afgelopen 3 maanden/Totaal (...)` → `mobile_total_pct`
- `Internet gebruik/Mobiel gebruik afgelopen 3 maanden/Mobiele telefoon of smart phone (...)` → `mobile_smartphone_pct`
- `Internet gebruik/Mobiel gebruik afgelopen 3 maanden/Laptop (...)` → `mobile_laptop_pct`
- `Internet gebruik/Mobiel gebruik afgelopen 3 maanden/Tablet (...)` → `mobile_tablet_pct`
- `Internet gebruik/Mobiel gebruik afgelopen 3 maanden/Andere mobiele apparatuur (...)` → `mobile_other_pct`

All values remain **percentages of the Dutch population aged 12+**.



In [20]:
cbs_rename_map = {
    "Perioden": "year",

    device_cols_raw[0]: "mobile_total_pct",

    device_cols_raw[1]: "mobile_smartphone_pct",

    device_cols_raw[2]: "mobile_laptop_pct",

    device_cols_raw[3]: "mobile_tablet_pct",

    device_cols_raw[4]: "mobile_other_pct",
}

cbs_devices_wide = cbs_devices_wide.rename(columns=cbs_rename_map)

print("Columns after renaming:")
print(cbs_devices_wide.columns.tolist())

cbs_devices_wide.head()

Columns after renaming:
['year', 'mobile_total_pct', 'mobile_smartphone_pct', 'mobile_laptop_pct', 'mobile_tablet_pct', 'mobile_other_pct']


,year,mobile_total_pct,mobile_smartphone_pct,mobile_laptop_pct,mobile_tablet_pct,mobile_other_pct
0,2012,51.40,40.30,29.20,16.10,NaN
1,2013,60.70,53.60,24.30,19.90,1.60
2,2014,65.90,60.40,23.70,23.30,4.90
3,2015,71.10,66.60,24.80,25.20,5.10
4,2016,73.20,70.50,22.00,21.90,4.50


## 4. Check types and sumary statistics 

Before reshaping, confirm: 

- `year` is numeric (int)
- All device columns are numeric floats (percentages)
- There are not strange non-numeric values in the subset.

In [21]:
print(cbs_devices_wide.dtypes)
print("\nSumamry stats:")
cbs_devices_wide.describe(include="all")

year                       int64
mobile_total_pct         float64
mobile_smartphone_pct    float64
mobile_laptop_pct        float64
mobile_tablet_pct        float64
mobile_other_pct         float64
dtype: object

Sumamry stats:


,year,mobile_total_pct,mobile_smartphone_pct,mobile_laptop_pct,mobile_tablet_pct,mobile_other_pct
count,8.00,8.00,8.00,8.00,8.00,7.00
mean,2015.50,71.35,66.56,28.77,24.55,7.03
std,2.45,11.61,14.59,6.08,5.88,4.09
min,2012.00,51.40,40.30,22.00,16.10,1.60
25%,2013.75,64.60,58.70,24.15,21.40,4.70
50%,2015.50,72.15,68.55,27.00,24.25,5.10
75%,2017.25,81.88,79.25,33.12,26.90,10.00
max,2019.00,84.40,82.50,38.70,35.80,13.10


## 5. Reshape into tidy format:`year`, `device_category`, `share_pct`

For analysis and plotting, a long / tidy format: 

- One row per `(year device_category)`
- Single numeric column `share_pct`

Target schema:

- `year` - e.g. 2012 ... 2019
- `device_category` - one of: 
    -`"mobile_total"`
    - `"smartphone"`
    - `"laptop"`
    - `"tablet"`
    - `"other_mobile"`
- `share_pct` - percentage of persons (12+) using internet on that device type.

In [22]:
# 5.1 Melt wide -> long 
cbs_devices_long = cbs_devices_wide.melt(
    id_vars="year",
    var_name="device_raw",
    value_name="share_pct"
)

cbs_devices_long.head()

,year,device_raw,share_pct
0,2012,mobile_total_pct,51.40
1,2013,mobile_total_pct,60.70
2,2014,mobile_total_pct,65.90
3,2015,mobile_total_pct,71.10
4,2016,mobile_total_pct,73.20


In [23]:
device_category_map = {
    "mobile_total_pct": "mobile_total",
    "mobile_smartphone_pct": "smartphone",
    "mobile_laptop_pct": "laptop",
    "mobile_tablet_pct": "tablet",
    "mobile_other_pct": "other_mobile",
}

cbs_devices_long["device_category"] = cbs_devices_long["device_raw"].map(device_category_map)

# Drop the helper column now that we have a clean category
cbs_devices_long = cbs_devices_long[["year", "device_category", "share_pct"]].copy()


cbs_devices_long.head(10)

,year,device_category,share_pct
0,2012,mobile_total,51.40
1,2013,mobile_total,60.70
2,2014,mobile_total,65.90
3,2015,mobile_total,71.10
4,2016,mobile_total,73.20
5,2017,mobile_total,82.40
6,2018,mobile_total,81.70
7,2019,mobile_total,84.40
8,2012,smartphone,40.30
9,2013,smartphone,53.60


## 6. Device usage over time 

Verify if the trends are logical: 

- Mobile usage should generally increase over time 
- Smartphone use should be high and rising
- Laptops and tablets fluctuate but stay lower than smartphones 

In [24]:
trend_table = cbs_devices_long.pivot_table(
    index="year",
    columns="device_category",
    values="share_pct"
)

trend_table

device_category,laptop,mobile_total,other_mobile,smartphone,tablet
year,,,,,
2012,29.20,51.40,NaN,40.30,16.10
2013,24.30,60.70,1.60,53.60,19.90
2014,23.70,65.90,4.90,60.40,23.30
2015,24.80,71.10,5.10,66.60,25.20
2016,22.00,73.20,4.50,70.50,21.90
2017,38.70,82.40,13.10,79.20,35.80
2018,32.50,81.70,9.10,79.40,26.70
2019,35.00,84.40,10.90,82.50,27.50


## 7. Save cleaned, analysis-ready CBS device ta ble

This file will be used later to: 

- Compare CBS device usabe with kaggle set's device mix 
- Integrate external statistics into the Customer Behaviour Insight Engine.

In [25]:
processed_path = "../data/processed/cbs_digital_clean.csv"

cbs_devices_long.to_csv(processed_path, index=False)

print("Saved cleaned CBS device data to:", processed_path)

Saved cleaned CBS device data to: ../data/processed/cbs_digital_clean.csv


## 8. Summary of Notebook 6b

- Reloaded the raw CBS dataset from `data/external/cbs_digital_adoption.csv`
- Selected only the columns relevant for mobile / device internet usage: 
    - Total mobile use 
    - Smartphone, laptop, table and other mobile equipment
- Translated Dutch CBS column names into clear English analysis names: 
- Reshaped the wide wable into a tidy format.

This cleaned table is now ready to be: 

- Joined with Kaggle-based session / device data
- Used in plots and comparrisons 
- Reused in other notebooks wihtout having to repeat the CBS cleaning steps.